In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import panel as pn
import plotly.express as px
import os
from requests import api 
from dotenv import load_dotenv
from pycoingecko import CoinGeckoAPI


pn.extension("plotly")

## Get the national currencies for countries

In [2]:
country_currency = api.get('http://country.io/currency.json').json()

# countries2 = api.get('https://restcountries.eu/rest/v2/').json()
# countries2

In [3]:
df_country_currency = pd.DataFrame(columns=['Code', 'Curr_code'], dtype=object)
for key,value in country_currency.items():
    df_country_currency = df_country_currency.append(pd.Series(
    [
        key,
        value
    ], index= df_country_currency.columns ), ignore_index=True)

In [4]:
df_country_currency.set_index('Code',inplace=True)
df_country_currency.head()

,Curr_code
Code,
BD,BDT
BE,EUR
BF,XOF
BG,BGN
BA,BAM


# Get the countries data - geocodes, currency codes, etc, saved in a CSV - concap.csv

In [5]:
df_country_geocodes = pd.read_csv('Data/concap.csv', index_col=None)
df_country_geocodes.dropna(inplace=True)
df_country_geocodes.set_index('CountryCode', inplace=True)

# Concatenate the 2 DFs together to have the data in the foll, format:
CountryCode|CountryName|CapitalName|CapitalLatitude|CapitalLongitude|ContinentName|Curr_code
---|---|---|---|---|---|---|
AU|Australia|Canberra|-35.2667|149.133|Australia|AUD

In [6]:
df_country_data = pd.concat([df_country_geocodes, df_country_currency], axis='columns', join='inner')
df_country_data.loc['AU']

CountryName         Australia
CapitalName          Canberra
CapitalLatitude      -35.2667
CapitalLongitude      149.133
ContinentName       Australia
Curr_code                 AUD
Name: AU, dtype: object

In [7]:
# Saving the data to CSV, we can just start with the CSVs in the final project code
df_country_data.to_csv('Data/Country_data.csv')

## Just a test to display the data obtained so far

In [8]:
load_dotenv()
map_box_api = os.getenv("MAPBOX_PUBLIC")

# Set the Mapbox API
px.set_mapbox_access_token(map_box_api)

In [9]:
hover_data = {'ContinentName': False, 'CountryName':False, 'CapitalLatitude':False, 'CapitalLongitude':False, 'Curr_code':True}

map1 = px.scatter_mapbox(
    df_country_data,
    lat="CapitalLatitude",
    lon="CapitalLongitude",
    color="ContinentName",
    hover_name='CountryName',
    hover_data= hover_data, #['CapitalName', 'Curr_code' ],
    zoom = 1,
    mapbox_style='basic'
)
map1.show()

In [10]:
df_country_data.loc[df_country_data.Curr_code == 'DOP']

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,ContinentName,Curr_code
DO,Dominican Republic,Santo Domingo,18.466667,-69.9,North America,DOP


# Getting the crypto data - This will be used in the final project code

In [11]:
# form the list of fiat currencies - We need to use this to call the coingecko API
fiat = df_country_data.Curr_code.to_list()

In [12]:
cryp_curr_list = ['bitcoin','litecoin', 'ripple', 'ethereum']

In [13]:
gecko = CoinGeckoAPI()
crypto_prices = gecko.get_price(ids=cryp_curr_list, vs_currencies=fiat, include_market_cap=True, include_24hr_vol=True, include_24hr_change=True)

In [14]:
df_gecko_data = pd.DataFrame.from_dict(crypto_prices)
df_gecko_data.index = df_gecko_data.index.str.upper()
df_gecko_data.head()

,litecoin,bitcoin,ethereum,ripple
AED,4.791800e+02,1.200410e+05,8.004160e+03,2.300000e+00
AED_24H_CHANGE,-7.677664e+00,-5.685176e+00,-8.218512e+00,-6.207155e+00
AED_24H_VOL,8.058348e+09,9.506692e+10,8.302614e+10,7.883031e+09
AED_MARKET_CAP,3.208387e+10,2.252063e+12,9.349455e+11,1.061754e+11
ARS,1.251564e+04,3.135312e+06,2.090580e+05,6.003000e+01


In [15]:
df_circ_coins = pd.DataFrame(columns=['crypto', 'Circ_coins'], index=None)
for curr in cryp_curr_list:
    circ_coins = df_gecko_data.loc['USD_MARKET_CAP', curr] / df_gecko_data.loc['USD', curr]
    df_circ_coins = df_circ_coins.append(pd.Series([
        curr,
        (circ_coins / 1000000)
    ], index= df_circ_coins.columns), ignore_index= True)
df_circ_coins.set_index('crypto', inplace=True)


In [24]:
df_circ_coins = df_circ_coins.loc[ df_circ_coins.index != 'ripple']

In [25]:
import dash 

In [28]:
import hvplot.pandas
df_circ_coins.hvplot.bar()    #ylabel= { 'Circ_coins': 'Coins in circulation'}

ValueError: String 'ylabel' only takes a string value.

:Bars   [crypto]   (Circ_coins)

## We will be forming 3 DFs from the data received

1. df_cryp_prices: will have the currency codes as the index, columns as the crypto names, and will store the current prices

2. df_cryp_change: will have the currency codes as the index, columns as the crypto names, and will store the changes over 24Hours. There will also be a col for Average change

3. df_cryp_vol: will have the currency codes as the index, columns as the crypto names, and will store the volume over 24Hours. There will also be a col for Average volume

In [19]:
crypto_names = df_gecko_data.columns.to_list()

## Forming empty dataframes here

In [20]:
df_cryp_prices = pd.DataFrame(columns= (['Curr'] + crypto_names)).set_index('Curr')

cols_change =['Curr']
for a in crypto_names: cols_change += [f'{a}_chg']
cols_change += ['Score_chg']
df_cryp_change = pd.DataFrame(columns=cols_change)

cols_vol =['Curr']
for a in crypto_names:  cols_vol += [f'{a}_vol']
cols_vol += ['Score_vol']
df_cryp_vol = pd.DataFrame(columns= cols_vol)
df_cryp_vol.columns

Index(['Curr', 'litecoin_vol', 'bitcoin_vol', 'ethereum_vol', 'ripple_vol',
       'Score_vol'],
      dtype='object')

## Populating the dataframes

In [21]:
row_2 = []
for row in df_gecko_data.index:
    
    row_data = df_gecko_data.loc[row]
    
    if len(row) == 3:
        df_cryp_prices = df_cryp_prices.append(row_data)
        
    elif '_24H_CHANGE' in row:
        row_2 = [row[0:3]]
        row_2.extend(row_data)
        avg = np.average(row_data)
        row_2.extend([np.average(row_data)])
        
        df_cryp_change = df_cryp_change.append(pd.Series(row_2, index= df_cryp_change.columns), ignore_index=True)
        
    elif '_24H_VOL' in row:
        row_2 = [row[0:3]]
        row_2.extend(row_data)
        avg = np.average(row_data)
        row_2.extend([np.average(row_data)])
        
        df_cryp_vol = df_cryp_vol.append(pd.Series(row_2, index= df_cryp_vol.columns), ignore_index=True)

In [22]:
df_cryp_change.set_index('Curr', inplace= True)
df_cryp_vol.set_index('Curr', inplace= True)
print(df_cryp_prices.head())
print(df_cryp_change.head())
print(df_cryp_vol.head())

      litecoin     bitcoin   ethereum     ripple
Curr                                            
AED     479.18   120041.00    8004.16   2.300000
ARS   12515.64  3135312.00  209058.00  60.030000
AUD     175.72    44019.00    2935.15   0.842857
BDT   11091.05  2778433.00  185262.00  53.200000
BHD      49.18    12319.84     821.47   0.235893
      litecoin_chg  bitcoin_chg  ethereum_chg  ripple_chg  Score_chg
Curr                                                                
AED      -7.677664    -5.685176     -8.218512   -6.207155  -6.947127
ARS      -7.657796    -5.664879     -8.198760   -6.186970  -6.927101
AUD      -6.602996    -4.587314     -7.150139   -5.115369  -5.863954
BDT      -7.601074    -5.606933     -8.142370   -6.129345  -6.869930
BHD      -7.691132    -5.698935     -8.231901   -6.220838  -6.960702
      litecoin_vol   bitcoin_vol  ethereum_vol    ripple_vol     Score_vol
Curr                                                                      
AED   8.058348e+09  9.50

In [23]:
df_crypto_complete = pd.concat([df_cryp_prices, df_cryp_change, df_cryp_vol], axis='columns', join='inner')

In [24]:
print(df_crypto_complete.head())

         bitcoin   ethereum     ripple  litecoin  bitcoin_chg  ethereum_chg  \
Curr                                                                          
AED    119996.00    8007.21   2.300000    480.42    -5.819597     -8.297467   
ARS   3134503.00  209163.00  60.030000  12549.50    -5.783740     -8.262553   
AUD     43959.00    2933.34   0.841897    176.00    -4.753843     -7.259753   
BDT   2777386.00  185333.00  53.190000  11119.72    -5.741465     -8.221391   
BHD     12315.59     821.81   0.235867     49.31    -5.830338     -8.307926   

      ripple_chg  litecoin_chg  Score_chg   bitcoin_vol  ethereum_vol  \
Curr                                                                    
AED    -6.410008     -7.521551  -7.012156  9.505628e+10  8.287117e+10   
ARS    -6.374376     -7.486342  -6.976753  2.483043e+12  2.164746e+12   
AUD    -5.350936     -6.475057  -5.959897  3.482267e+10  3.035881e+10   
BDT    -6.332366     -7.444831  -6.935013  2.200148e+12  1.918115e+12   
BHD    -

## Forming the dataframe which can be sent for plotting

In [25]:
col_names_plotting = ['Curr'] + crypto_names + ['Score_chg', 'Score_vol']
df_crypto_print = pd.DataFrame(columns= col_names_plotting)
df_crypto_print

,Curr,bitcoin,ethereum,ripple,litecoin,Score_chg,Score_vol


In [26]:
# Maybe we can use u'\u2191' in the price text to display up-arrow and u'\u2193' for down arrow

for row in df_crypto_complete.index:
    
    row_print = [row]
    
    for name in crypto_names:
        
        col_name = name
        
        price_val = df_crypto_complete.loc[row][col_name]
        
        change_val = df_crypto_complete.loc[row][f'{col_name}_chg']
        
        vol_val = df_crypto_complete.loc[row][f'{col_name}_vol']
        
        price = "Price:  {:,.2f}".format(price_val) + ", Change: {:,.2f}".format(change_val) + ", Volume: {:,.2f}".format(vol_val)
        
        row_print += [price]
        # print(type(vol_val))
        
        
    row_print += [df_crypto_complete.loc[row]['Score_chg']] + [df_crypto_complete.loc[row]['Score_vol']] 
#     print(row_print)
    df_crypto_print = df_crypto_print.append(pd.Series(row_print, index= df_crypto_print.columns), ignore_index=True)

In [27]:
df_crypto_print.set_index('Curr', inplace= True)

In [28]:
print(df_crypto_print.head())

                                                bitcoin  \
Curr                                                      
AED   Price:  119,996.00, Change: -5.82, Volume: 95,...   
ARS   Price:  3,134,503.00, Change: -5.78, Volume: 2...   
AUD   Price:  43,959.00, Change: -4.75, Volume: 34,8...   
BDT   Price:  2,777,386.00, Change: -5.74, Volume: 2...   
BHD   Price:  12,315.59, Change: -5.83, Volume: 9,75...   

                                               ethereum  \
Curr                                                      
AED   Price:  8,007.21, Change: -8.30, Volume: 82,87...   
ARS   Price:  209,163.00, Change: -8.26, Volume: 2,1...   
AUD   Price:  2,933.34, Change: -7.26, Volume: 30,35...   
BDT   Price:  185,333.00, Change: -8.22, Volume: 1,9...   
BHD   Price:  821.81, Change: -8.31, Volume: 8,505,3...   

                                                 ripple  \
Curr                                                      
AED   Price:  2.30, Change: -6.41, Volume: 7,881,953..

In [29]:
df_plot = df_country_data.merge(df_crypto_print, left_on='Curr_code', right_index=True, how='inner')

In [30]:
df_plot.head()

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,ContinentName,Curr_code,bitcoin,ethereum,ripple,litecoin,Score_chg,Score_vol
GS,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,Antarctica,GBP,"Price: 23,709.00, Change: -5.60, Volume: 18,7...","Price: 1,582.09, Change: -8.08, Volume: 16,37...","Price: 0.45, Change: -6.19, Volume: 1,557,340...","Price: 94.92, Change: -7.31, Volume: 1,594,47...",-6.794592,9.576815e+09
GG,Guernsey,Saint Peter Port,49.450000,-2.533333,Europe,GBP,"Price: 23,709.00, Change: -5.60, Volume: 18,7...","Price: 1,582.09, Change: -8.08, Volume: 16,37...","Price: 0.45, Change: -6.19, Volume: 1,557,340...","Price: 94.92, Change: -7.31, Volume: 1,594,47...",-6.794592,9.576815e+09
IM,Isle of Man,Douglas,54.150000,-4.483333,Europe,GBP,"Price: 23,709.00, Change: -5.60, Volume: 18,7...","Price: 1,582.09, Change: -8.08, Volume: 16,37...","Price: 0.45, Change: -6.19, Volume: 1,557,340...","Price: 94.92, Change: -7.31, Volume: 1,594,47...",-6.794592,9.576815e+09
JE,Jersey,Saint Helier,49.183333,-2.100000,Europe,GBP,"Price: 23,709.00, Change: -5.60, Volume: 18,7...","Price: 1,582.09, Change: -8.08, Volume: 16,37...","Price: 0.45, Change: -6.19, Volume: 1,557,340...","Price: 94.92, Change: -7.31, Volume: 1,594,47...",-6.794592,9.576815e+09
GB,United Kingdom,London,51.500000,-0.083333,Europe,GBP,"Price: 23,709.00, Change: -5.60, Volume: 18,7...","Price: 1,582.09, Change: -8.08, Volume: 16,37...","Price: 0.45, Change: -6.19, Volume: 1,557,340...","Price: 94.92, Change: -7.31, Volume: 1,594,47...",-6.794592,9.576815e+09


In [31]:
hover_data = {'ContinentName': False, 'CountryName':False, 'CapitalLatitude':False, 'CapitalLongitude':False, 'Curr_code':True,
              'ripple': True, 'bitcoin': True, 'litecoin':True, 'ethereum': True}

map1 = px.scatter_mapbox(
    df_plot,
    lat="CapitalLatitude",
    lon="CapitalLongitude",
    color="Score_chg",
    hover_name='CountryName',
    hover_data= hover_data, #['CapitalName', 'Curr_code' ],
    zoom = 2,
    mapbox_style='basic',
    height=1000
)
map1.show()

## Plotting the amount of coins in circulation

In [53]:
# Get the data for USD
df_circ_coins = pd.DataFrame(columns=['crypto', 'Circ_coins'], index=None)
for curr in cryp_curr_list:
    circ_coins = df_gecko_data.loc['USD_MARKET_CAP', curr] / df_gecko_data.loc['USD', curr]
    df_circ_coins = df_circ_coins.append(pd.Series([
        curr,
        (circ_coins / 1000000)
    ], index= df_circ_coins.columns), ignore_index= True)
# df_circ_coins.set_index('crypto', inplace=True)

In [54]:
df_circ_coins

,crypto,Circ_coins
0,bitcoin,18.767763
1,litecoin,66.782101
2,ripple,46200.486020
3,ethereum,116.762908


In [45]:
import hvplot.pandas


In [55]:
df_circ_coins.hvplot.bar(x= 'crypto', y= 'Circ_coins', rot= 90)

:Bars   [crypto]   (Circ_coins)